참고 사이트 https://wikidocs.net/159468

In [2]:
import numpy as np
import pandas as pd
import itertools
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
local_mealion_data = pd.read_csv('C:/Users/junseok/Downloads/local_mealion_data.csv')
doc = local_mealion_data['content'][1700]
doc

'[ 가로수길 📍소각 ]\n-\n🍽#마라샹궈 (19,900)\n-\n마라초심자들은 저장하세요... 먹어본 마라샹궈 중 가장 깔끔하면서 자극적이지 않고... 근데 맛있는!!\n마라샹궈 외 지삼선, 마파두부, 꿔바로우 등 메뉴가 다양한데 평일 런치에는 저렴한 가격으로 식사가 가능하다😆\n런치 마라샹궈 주문했는데 처음에 2만원이란 가격에 비싸다 싶었지만 들어가 있는 재료 보니 납득!\n재료를 고를 수 없는게 아쉽긴 하지만 있을거 다 있고 심지어 피쉬볼, 어묵, 새우 같은 토핑까지!!\n팁을 드리자면 청경채 추가하고 잘먹는 편이면 라면사리 추가도 추천😋\n마라 완전 초심자는 1단계, 매운맛에 약한 마라쟁이 or 적응해 가는 마라초심자는 2단계, 마라중독자는 3단계로~~\n\n매장 깔끔하고 종업원 분 너무너무 친절하시고, 셀프바에 고수 알아서 가져갈 수 있는거 너무 맘에 든다🤩'

In [4]:
okt = Okt()
tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('[', 'Punctuation'), ('가로수길', 'Noun'), ('📍', 'Foreign'), ('소각', 'Noun'), (']', 'Punctuation'), ('\n', 'Foreign'), ('-', 'Punctuation'), ('\n', 'Foreign'), ('🍽#마라샹궈', 'Hashtag'), ('(', 'Punctuation')]
명사 추출 : 가로수길 소각 초심자 저장 마라샹궈 중 가장 자극 마라샹궈 외 삼선 마파두부 바 로우 등 메뉴 평일 런치 가격 식사 런치 마라샹궈 주문 처음 가격 재료 납득 재료 수 심지어 피쉬 볼 어묵 새우 토핑 팁 청경채 추가 편이 라면 사리 추가 추천 완전 초심자 단계 맛 마라쟁 초심자 단계 중독 단계 매장 종업원 분 셀프 바 고수 수 맘


In [5]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 92
trigram 다섯개만 출력 : ['가격 식사', '가격 식사 런치', '가격 재료', '가격 재료 납득', '가로수길 소각']


In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [7]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['가격 재료 납득', '메뉴 평일', '새우 토핑 청경채', '메뉴 평일 런치', '마파두부 로우 메뉴']


In [8]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [9]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['새우 토핑', '가격 재료', '평일 런치 가격', '새우 토핑 청경채', '마파두부 로우 메뉴']

In [10]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['종업원 셀프 고수', '마라쟁 초심자', '가격 재료', '새우 토핑 청경채', '마파두부 로우 메뉴']

In [11]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
    
    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [12]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['마파두부 로우 메뉴', '새우 토핑 청경채', '가격 재료 납득', '메뉴 평일 런치', '평일 런치 가격']

In [13]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['마파두부 로우 메뉴', '가로수길 소각', '종업원 셀프 고수', '초심자 단계 중독', '새우 토핑 청경채']

In [14]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.85)

['마파두부 로우 메뉴', '가로수길 소각', '종업원 셀프 고수', '초심자 단계 중독', '새우 토핑 청경채']

In [15]:
import time
import pandas as pd

local_mealion_data = pd.read_csv('C:/Users/junseok/Downloads/local_mealion_data.csv')
local_mealion_data["keyword_mmr_7_topn_5"] = ""
local_mealion_data["keyword_mmr_85_topn_5"] = ""
local_mealion_data["keyword_max_sum_sim_ngram_13_topn_5"] = ""
local_mealion_data["keyword_max_sum_sim_ngram_13_topn_3"] = ""
local_mealion_data


,id,content,summary,wouldVisitAgain,authorId,mentioningReiviewId,restaurantId,menus,published,feedback,...,keyword_mmr_0.85_topn_5,lon_lat,lon,lat,tag,comnbine_tag_keywords,restaurant_name,comnbine_tag_review,keyword_mmr_7_topn_5,keyword_mmr_85_topn_5
0,2,일단 이모들이 너무 친절하고 에너지 넘쳐요!\n반찬은 대체적으로 짠 편이에요ㅜ\n생...,NaN,0,3.0,NaN,385515,맛있는생선구이백반,1,NaN,...,"['반찬 체적 편이', '생선', '이모', '에너지', '임연 수로']","(127.728089533208, 37.8830834006352)",127.728090,37.883083,NaN,생선 일단 에너지 반찬 체적 이모 체적 편이 생선 맛있는생선구이백반,강릉집,일단 이모들이 너무 친절하고 에너지 넘쳐요 반찬은 대체적으로 짠 편이에요ㅜ 생선은...,,
1,3,항상 싸고 맛있게 먹을 수 있다. 다른데가 물가오르면서 다 메뉴가격을 올린반면에 길...,NaN,0,1.0,NaN,514251,길낙지볶음,1,NaN,...,"['항상 물가 메뉴', '유지 남춘천역 접근성', '물가', '접근성 기회', '메뉴']","(127.722357089802, 37.8656621498275)",127.722357,37.865662,NaN,유지 유지 남춘천역 접근성 물가 접근성 기회 메뉴 길낙지볶음,춘천길낙지,항상 싸고 맛있게 먹을 수 있다 다른데가 물가오르면서 다 메뉴가격을 올린반면에 ...,,
2,4,강원대 체육관 쪽문으로 내려오면 바로 보여요!\n일단 양이 엄~청 많아요!\n게다가...,NaN,0,3.0,NaN,341523,"짜장면,짬뽕,미니탕수육",1,NaN,...,"['게다가 짜장면 고기', '체육관 쪽문', '최고', '강원 체육관', '미니 탕...","(127.7393637, 37.8707727)",127.739364,37.870773,NaN,체육관 쪽문 최고 강원 체육관 미니 탕수육 가성 고기 탕수육 짜...,죽향,강원대 체육관 쪽문으로 내려오면 바로 보여요 일단 양이 엄 청 많아요 게다가 짜장...,,
3,9,투썸은 조각케잌 필수인거 다들 아시죠?!!\n제 최애는 티라미수에요\n먹을때마다 기...,NaN,0,6.0,NaN,314108,"티라미수,아메리카노",1,NaN,...,"['조각 케잌', '기침 자꾸', '자꾸', '조각', '최애 티라미수']","(127.753314435589, 37.9039346405307)",127.753314,37.903935,NaN,조각 아시 기침 기침 자꾸 조각 케잌 티라미수 아메리카노,투썸플레이스,투썸은 조각케잌 필수인거 다들 아시죠 제 최애는 티라미수에요 먹을때마다 기침이 나...,,
4,11,저렴한 가격으로 배를 채울 수 있는 곳!\n대표메뉴인 청와대 순두부에요\n비빔밥과 ...,NaN,0,3.0,NaN,227919,청와대순두부,1,NaN,...,"['청와대 순두부 비빔밥', '가격', '청와대', '순두부 무려 지하도', '비빔밥']","(127.745073540877, 37.8732551401682)",127.745074,37.873255,NaN,가격 청와대 청와대 순두부 순두부 비빔밥 비빔밥 순두부 청와대순두부,봉실스넥2,저렴한 가격으로 배를 채울 수 있는 곳 대표메뉴인 청와대 순두부에요 비빔밥과 순두...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806,9519,"[ 보광동 📍서가네왕솥뚜껑삼겹살 ]\n-\n🐷삼겹살 (200g 12,000)\n🐷목...",보광동 삼겹살은 여기!,1,1043.0,NaN,922387,NaN,1,NaN,...,"['한남동 분위기 보광동', '상여 남자 고기', '먹음 둘이서 한남동', '비만'...","(126.9968094386, 37.5310332878)",126.996809,37.531033,"[[['고깃집', '활기찬', '노포', '친구와', '부모님과', '저녁']]]",고깃집 활기찬 노포 친구와 부모님과 저녁 보광동 도망 한남동...,서가네왕솥뚜껑삼겹살,고깃집 활기찬 노포 친구와 부모님과 저녁 보광동 서가네왕...,,
1807,9520,"[ 신사 📍카바동 ]\n-\n🍽타레카츠동 (11,000)\n🍽가라이규동 (11,50...",두툼 바삭한 돈까스를 덮밥으로,0,1043.0,9512.0,936270,NaN,1,NaN,...,"['달콤 레소스 계란', '고기 라면', '규동', '기름 추가 유독', '컵누 신...","(127.0243552, 37.5173078)",127.024355,37.517308,"[[['일식', '활기찬', '이국적인', '친구와', '연인과', '점심', '저...",일식 활기찬 이국적인 친구와 연인과 점심 저녁 짭쪼름한 ...,카바동,일식 활기찬 이국적인 친구와 연인과 점심 저녁 신사 ...,,
1808,9521,포시즌스 호텔의 신상 바 OUL (오울)\n예전에 보칼리노 와인바 였던 곳이 한국식...,"한국식 바, 포시즌스 호텔 오울!",0,1094.0,NaN,936264,NaN,1,NaN,...,"['비쥬 상큼 과일', '차돌박이 화채 폭탄주', '직원 한복 유니폼', '얼음 가...","(126.9753951, 37.5706979)",126.975395,37.570698,"[[['한식', '술집', '활기찬', '특색있는', '친구와', '연인과', '저...",한식 술집 활기찬 특색있는 친구와 연인과 저녁 특별한날 ...,포시즌스호텔서울 오울,한식 술집 활기찬 특색있는 친구와 연인과 저녁 특별한날 ...,,
1809,9522,🍽화성동탄 - [ #스시코호시 ] 4.3/5.0\n⏰ 점심 1부 12:00~13:3...,가격대비 다양한 음식 맛은 글쎄,0,1063.0,NaN,936281,NaN,1,NaN,...,"['음식 차완무시 시작', '화성', '제철 재료', '와사비 후토 마키', '보완...","(127.0717445, 37.2040208)",127.071744,37.204021,"[[['일식', '깔끔한', '조용한', '고급스러운', '연인과', '부모님과',...",일식 깔끔한 조용한 고급스러운 연인과 부모님과 비즈니스 ...,스시코호시 동탄,일식 깔끔한 조용한 고급스러운 연인과 부모님과 비즈니스 ...,,


In [ ]:
for i in range(0,1812):

    try:
        if i % 100 == 0 or i % 1811 == 0:
            local_mealion_data.to_csv('C:/Users/junseok/Downloads/local_mealion_data.csv', index=False)
        print(i)
        doc = local_mealion_data['content'][i]
        
        okt = Okt()
        tokenized_doc = okt.pos(doc)
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

        n_gram_range = (1, 3)
        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names()
        
        time.sleep(1)
        model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
        doc_embedding = model.encode([doc])
        candidate_embeddings = model.encode(candidates)
        time.sleep(1)
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        try:
            keyword_mmr_7_topn_5 =mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)
        except:
            keyword_mmr_7_topn_5 = ''
            pass
        try:
            keyword_mmr_85_topn_5 = mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.85)
        except:
            keyword_mmr_85_topn_5 = ''
            pass
        try:
            keyword_max_sum_sim_ngram_13_topn_5 = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)
        except: 
            keyword_max_sum_sim_ngram_13_topn_5 = ''
            pass
        try:
            
            keyword_max_sum_sim_ngram_13_topn_3 = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=30)
        except: 
            keyword_max_sum_sim_ngram_13_topn_3
            pass

    except:
        continue
    local_mealion_data["keyword_mmr_7_topn_5"][i] = keyword_mmr_7_topn_5
    local_mealion_data["keyword_mmr_85_topn_5"][i] = keyword_mmr_85_topn_5
    local_mealion_data["keyword_max_sum_sim_ngram_13_topn_5"][i] = keyword_max_sum_sim_ngram_13_topn_5
    local_mealion_data["keyword_max_sum_sim_ngram_13_topn_3"][i] = keyword_max_sum_sim_ngram_13_topn_3
    
    



In [ ]:
import pandas as pd
mealion_review = pd.read_csv('C:/Users/junseok/Downloads/mealion_review.csv')


In [ ]:
 mealion_review[mealion_review['review_menu'].isnull()]

In [126]:
mealion_review.to_csv('C:/Users/junseok/Downloads/mealion_review.csv', index=False)
mealion_review

,review_id,review_content,review_title,3,4,5,retaurant_id,review_menu,8,9,10,11,address,city,province,keyword_max_sum_sim_ngram_13_topn_3,keyword_max_sum_sim_ngram_24_topn_3
0,2,일단 이모들이 너무 친절하고 에너지 넘쳐요!\n반찬은 대체적으로 짠 편이에요ㅜ\n생...,NaN,0,3.0,NaN,385515,맛있는생선구이백반,1,NaN,2020-11-25 21:23:26,2020-11-27 20:36:12,강원도 춘천시 서부대성로 46,강원도 춘천시,강원도,"['생선', '에너지 반찬 체적', '이모']","[에너지 반찬 체적, 일단 이모, 체적 편이 생선 임연]"
1,3,항상 싸고 맛있게 먹을 수 있다. 다른데가 물가오르면서 다 메뉴가격을 올린반면에 길...,NaN,0,1.0,NaN,514251,길낙지볶음,1,NaN,2020-11-25 21:23:44,2020-11-25 21:28:18,강원도 춘천시 우묵길 52,강원도 춘천시,강원도,"['유지', '유지 남춘천역 접근성', '물가']","[유지 남춘천역 접근성, 접근성 기회, 물가 메뉴]"
2,4,강원대 체육관 쪽문으로 내려오면 바로 보여요!\n일단 양이 엄~청 많아요!\n게다가...,NaN,0,3.0,NaN,341523,"짜장면,짬뽕,미니탕수육",1,NaN,2020-11-25 21:49:39,2020-11-25 21:53:47,강원도 춘천시 효제길 55,강원도 춘천시,강원도,"['체육관 쪽문', '최고', '고기 탕수육']","[강원 체육관, 바로 일단 게다가 짜장면, 고기 탕수육]"
3,9,투썸은 조각케잌 필수인거 다들 아시죠?!!\n제 최애는 티라미수에요\n먹을때마다 기...,NaN,0,6.0,NaN,314108,"티라미수,아메리카노",1,NaN,2020-11-25 22:23:16,2020-11-25 22:25:07,강원도 춘천시 동면 춘천순환로 734,강원도 춘천시,강원도,"['아시', '기침 자꾸', '조각 케잌']","[아시 최애, 기침 자꾸, 조각 케잌]"
4,11,저렴한 가격으로 배를 채울 수 있는 곳!\n대표메뉴인 청와대 순두부에요\n비빔밥과 ...,NaN,0,3.0,NaN,227919,청와대순두부,1,NaN,2020-11-25 22:28:38,2020-11-25 22:30:46,강원도 춘천시 서부대성로239번길 8,강원도 춘천시,강원도,"['가격', '청와대', '순두부 비빔밥']","[가격 대표, 청와대 순두부, 순두부 비빔밥]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,9519,"[ 보광동 📍서가네왕솥뚜껑삼겹살 ]\n-\n🐷삼겹살 (200g 12,000)\n🐷목...",보광동 삼겹살은 여기!,1,1043.0,NaN,922387,NaN,1,NaN,2022-04-23 10:23:42,2022-04-23 10:23:42,서울특별시 용산구 우사단로 8,서울특별시 용산구,서울특별시,"['보광동 도망 한남동', '보광동 사실 젊음', '고기 채소 무쌈']","[국내 삼겹 목살, 된장찌개 직접 김치 파절, 주문 국내 삼겹 목살]"
1836,9520,"[ 신사 📍카바동 ]\n-\n🍽타레카츠동 (11,000)\n🍽가라이규동 (11,50...",두툼 바삭한 돈까스를 덮밥으로,0,1043.0,9512.0,936270,NaN,1,NaN,2022-04-24 00:36:07,2022-04-24 00:36:07,서울특별시 강남구 논현로149길 52 1층 카바동,서울특별시 강남구,서울특별시,"['고기 라면', '계란 조화', '테이블 짭쪼름한 은근']","[고기 라면 가라이 규동, 계란 이불, 테이블 짭쪼름한 은근]"
1837,9521,포시즌스 호텔의 신상 바 OUL (오울)\n예전에 보칼리노 와인바 였던 곳이 한국식...,"한국식 바, 포시즌스 호텔 오울!",0,1094.0,NaN,936264,NaN,1,NaN,2022-04-24 09:14:40,2022-04-24 09:16:30,서울특별시 종로구 새문안로 97 포시즌스호텔서울 2층,서울특별시 종로구,서울특별시,"['폭탄주 한국', '과일', '한식 안주 메뉴']","[한식 안주 메뉴, 폭탄주 한국 재료, 상큼 과일]"
1838,9522,🍽화성동탄 - [ #스시코호시 ] 4.3/5.0\n⏰ 점심 1부 12:00~13:3...,가격대비 다양한 음식 맛은 글쎄,0,1063.0,NaN,936281,NaN,1,NaN,2022-04-25 13:42:01,2022-04-25 13:42:01,경기도 화성시 동탄반석로 172 B102동 2층 227호,경기도 화성시,경기도,"['간이 초밥 생선', '접객', '제철 재료 음식']","[차완무시 시래기 샐러드, 접객 서버 접객, 제철 재료 음식 노력]"


In [21]:
local_mealion_data.iloc[0]

id                                                                                     2
content                                일단 이모들이 너무 친절하고 에너지 넘쳐요!\n반찬은 대체적으로 짠 편이에요ㅜ\n생...
summary                                                                              NaN
wouldVisitAgain                                                                        0
authorId                                                                             3.0
mentioningReiviewId                                                                  NaN
restaurantId                                                                      385515
menus                                                                          맛있는생선구이백반
published                                                                              1
feedback                                                                             NaN
createdAt                                                            2020-11-25 21:23:26
updatedAt            